In [1]:
!cp /kaggle/input/bengali-eval-data/predict.py .

!cp -r ../input/python-packages2 ./
!tar xvfz ./python-packages2/jiwer.tgz
!pip install ./jiwer/python-Levenshtein-0.12.2.tar.gz -f ./ --no-index
!pip install ./jiwer/jiwer-2.3.0-py3-none-any.whl -f ./ --no-index

cp: cannot stat '/kaggle/input/bengali-eval-data/predict.py': No such file or directory
jiwer/
jiwer/jiwer-2.3.0-py3-none-any.whl
jiwer/python-Levenshtein-0.12.2.tar.gz
jiwer/setuptools-65.3.0-py3-none-any.whl
Looking in links: ./
Processing ./jiwer/python-Levenshtein-0.12.2.tar.gz
  Preparing metadata (setup.py) ... - \ done
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp310-cp310-linux_x86_64.whl size=79704 sha256=f1c058b22f21030cd9aa89906c949bf04fdc37e8be736494ca469403a0f60c74
  Stored in directory: /root/.cache/pip/wheels/f1/ab/b1/90d2068d73d15e52c1a65676d269a9f043b61221a29f7298e7
Successfully built python-Levenshtein
  Attempting uninstall: python-Levenshtein
    Found existing installation: python-Levenshtein 0.21.1
    Uninstalling python-Levenshtein-0.21.1:
      Successfully uninstalled python-Levenshtein-0.21.1
Looking in links: ./
Processing ./jiwer/jiwer-2.3.0-py3-none-any.whl


In [2]:
import os
import csv
import time
import glob

MODEL = '/kaggle/input/bengali-ai-asr-submission/bengali-whisper-medium/'
PUNCT_MODELS = [
    '/kaggle/input/bengali-ai-asr-submission/punct-model-6layers/',
    '/kaggle/input/bengali-ai-asr-submission/punct-model-8layers/',
    '/kaggle/input/bengali-ai-asr-submission/punct-model-11layers/',
    '/kaggle/input/bengali-ai-asr-submission/punct-model-12layers/'
]
CHUNK_LENGTH_S = 20.1
ENABLE_BEAM = True
# none alone 0.9 0.037964275588318684
# none alone 0.7 0.03592288063510065
# none alone 0.4 0.03416501275871846
PUNCT_WEIGHTS = [[1.0, 1.4, 1.0, 0.8]]

if ENABLE_BEAM:
    BATCH_SIZE = 4
else:
    BATCH_SIZE = 8

if len(glob.glob("/kaggle/input/bengaliai-speech/test_mp3s/*.mp3")) > 10:
    EVAL = False
    DATASET_PATH = '/kaggle/input/bengaliai-speech/test_mp3s/'
else:
    EVAL = True
    DATASET_PATH = '/kaggle/input/bengaliai-speech/test_mp3s/'
    
import csv
import glob
import shutil
import librosa
import argparse
import warnings
from pathlib import Path
import transformers
print(transformers.__version__)
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer

import warnings

warnings.filterwarnings("ignore")

files = list(glob.glob(DATASET_PATH + '/' + '*.wav'))
files += list(glob.glob(DATASET_PATH + '/' + '*.mp3'))
files.sort()

pipe = pipeline(task="automatic-speech-recognition",
                model=MODEL,
                tokenizer=MODEL,
                chunk_length_s=CHUNK_LENGTH_S, device=0, batch_size=BATCH_SIZE)
pipe.model.config.forced_decoder_ids = pipe.tokenizer.get_decoder_prompt_ids(language="bn", task="transcribe")

print("model loaded!")

4.30.2


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

model loaded!


In [3]:
def fix_repetition(text, max_count):
    uniq_word_counter = {}
    words = text.split()
    for word in text.split():
        if word not in uniq_word_counter:
            uniq_word_counter[word] = 1
        else:
            uniq_word_counter[word] += 1

    for word, count in uniq_word_counter.items():
        if count > max_count:
            words = [w for w in words if w != word]
    text = " ".join(words)
    return text

In [4]:
if ENABLE_BEAM:
    texts = pipe(files, generate_kwargs={"max_length": 260, "num_beams": 4})
else:
    texts = pipe(files)

In [5]:
del pipe
import torch
models = [
    AutoModelForTokenClassification.from_pretrained(f).eval().cuda() for f in PUNCT_MODELS
]
tokenizer = AutoTokenizer.from_pretrained(PUNCT_MODELS[0])
def punctuate(text):
    input_ids = tokenizer(text).input_ids
    with torch.no_grad():
        model = models[0]
        logits = torch.nn.functional.softmax(
            model(input_ids=torch.LongTensor([input_ids]).cuda()).logits[0, 1:-1],
            dim=1).cpu()
        for model in models[1:]:
            logits += torch.nn.functional.softmax(
                model(input_ids=torch.LongTensor([input_ids]).cuda()).logits[0, 1:-1],
                dim=1).cpu()
        logits = logits / len(models)
        logits *= torch.FloatTensor(PUNCT_WEIGHTS)
        label_ids = torch.argmax(logits, dim=-1)

        tokens = tokenizer(text, add_special_tokens=False).input_ids
        punct_text = ""
        for index, token in enumerate(tokens):
            token_str = tokenizer.decode(token)
            if '##' not in token_str:
                punct_text += " " + token_str
            else:
                punct_text += token_str[2:]
            punct_text += ['', '।', ',', '?'][label_ids[index].item()]

    punct_text = punct_text.strip()
    return punct_text

In [6]:
predictions = []
with open("submission.csv", 'wt', encoding="utf8") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['id', 'sentence'])
    for f, text in zip(files, texts):
        file_id = Path(f).stem
        pred = text['text'].strip()
        pred = fix_repetition(pred, max_count=8)
        pred = punctuate(pred)
        if pred[-1] not in ['।', '?', ',']:
            pred = pred + '।'
        # print(i, file_id, pred)
        prediction = [file_id, pred]
        writer.writerow(prediction)
        predictions.append(prediction)
print("inference finished!")

inference finished!
